# Electronic Parts Problem

Cristina Hernández, Beatriz Jimenez, Macarena Vargas, Guillermo Ruiz

In [29]:
import pyomo.environ as pe
import pyomo.opt as po
from pyomo.environ import NonNegativeReals, Binary

#### Create the model 

In [30]:
model = pe.ConcreteModel()

#### Sets


$e$: electronic parts {A,B,C}

$l$: production lines {L1,L2,L3}

$r$: robots {R1,R2}

$a$: attribute {profit, production}

In [31]:
model.e = pe.Set(initialize=['A', 'B', 'C'])                  
model.l = pe.Set(initialize=['L1', 'L2', 'L3'])               
model.r = pe.Set(initialize=['R1', 'R2'])                     
model.a = pe.Set(initialize=['profit', 'production'])

#### Parameters
 $Q_e$: production capacity of any line for part e [units/week]

 $P_e$: unitary profit of part e [€/unit]

 $AQ_r$: increased capacity for line if robot r is used [units/week]

 $C_r$: cost of robot r [€/week]

 $Wa$: weight for attribute a (can be set to 1 for both if you want equal importance)

 $GOAL_a$:  goal for each attribute (12000€/week for profit, 1650 units/week for production)


In [32]:
model.Q = pe.Param(model.e, initialize={'A': 200, 'B': 300, 'C': 400})

model.P = pe.Param(model.e, initialize={'A': 15, 'B': 12, 'C': 10})


model.AQ = pe.Param(model.r, initialize={'R1': 100, 'R2': 150})


model.C = pe.Param(model.r, initialize={'R1': 1000, 'R2': 1400})


model.W = pe.Param(model.a, initialize={'profit': 1, 'production': 1})

model.GOAL = pe.Param(model.a, initialize={'profit': 12000, 'production': 1650})

#### Variables
$x_{e,l}$: produced units of part e in line l [units/week]

$z_{e,l}$: assignment of part e to line l (binary, 1 if assigned, 0 otherwise)

$u_{r,l}$: hire of robot r to line l (binary, 1 if robot r is installed in line l)

$p_a$: positive deviation from attribute a (profit, production)

$n_a$: negative deviation from attribute a (profit, production)


In [33]:
model.x = pe.Var(model.e, model.l, domain=NonNegativeReals)


model.z = pe.Var(model.e, model.l, domain=Binary)

model.u = pe.Var(model.r, model.l, domain=Binary)

model.p = pe.Var(model.a, domain=NonNegativeReals)

model.n = pe.Var(model.a, domain=NonNegativeReals)

#### Objective Function

$\min \sum_{a} W_{a} \cdot n_{a}$

In [34]:

def objective_rule(model):
    return sum(model.W[a] * model.n[a] for a in model.a)

model.obj = pe.Objective(rule=objective_rule, sense=pe.minimize)

#### Constraints

Goal for profits [€/week]

$\sum_{e,l} P_e x_{e,l} - \sum_{r,l} C_r u_{r,l} + n_{\text{profit}} - p_{\text{profit}} = GOAL_{\text{profit}}$


In [35]:

def profit_goal_rule(model):
    return sum(model.P[e] * model.x[e, l] for e in model.e for l in model.l) \
           - sum(model.C[r] * model.u[r, l] for r in model.r for l in model.l) \
           + model.n['profit'] - model.p['profit'] == model.GOAL['profit']

model.profit_goal = pe.Constraint(rule=profit_goal_rule)

Goal for production [units]

$\sum_{e,l} x_{e,l} + n_{\text{production}} - p_{\text{production}} = GOAL_{\text{production}}$


In [36]:

def prod_goal_rule(model):
    return sum(model.x[e, l] for e in model.e for l in model.l) \
           + model.n['production'] - model.p['production'] == model.GOAL['production']

model.prod_goal = pe.Constraint(rule=prod_goal_rule)

Assignment of part e to production line l

$x_{e,l} \leq M \cdot z_{e,l} \qquad \forall\, e,\, l$


In [37]:
M = max(model.Q[e] for e in model.e) + max(model.AQ[r] for r in model.r)

def assign_rule(model, e, l):
    return model.x[e, l] <= M * model.z[e, l]
model.assign = pe.Constraint(model.e, model.l, rule=assign_rule)

Maximum production capacity of part e in production line l

$x_{e,l} \leq Q_e z_{e,l} + \sum_{r} AQ_r u_{r,l} \qquad \forall\, e,\, l$

In [38]:
def capacity_rule(model, e, l):
    return model.x[e, l] <= model.Q[e] * model.z[e, l] + sum(model.AQ[r] * model.u[r, l] for r in model.r)
model.capacity = pe.Constraint(model.e, model.l, rule=capacity_rule)

One product  per line

$\sum_{e} z_{e,l} = 1 \qquad \forall\, l$

In [39]:
def one_product_per_line_rule(model, l):
    return sum(model.z[e, l] for e in model.e) == 1
model.one_product_per_line = pe.Constraint(model.l, rule=one_product_per_line_rule)

One robot per line

$\sum_{r} u_{r,l} \leq 1 \qquad \forall\, l$

In [40]:
def one_robot_per_line_rule(model, l):
    return sum(model.u[r, l] for r in model.r) <= 1
model.one_robot_per_line = pe.Constraint(model.l, rule=one_robot_per_line_rule)

No puede haber más de 1 robot de cada tipo en todo el sistema

$\sum_{l} u_{r, l} \leq 1 \qquad \forall\, r$

In [41]:

def robot_limit_rule(model, r):
    return sum(model.u[r, l] for l in model.l) <= 1
model.robot_limit = pe.Constraint(model.r, rule=robot_limit_rule)

#### Solve model

In [42]:
solver = pe.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

# Mostrar estado de la optimización
print("Solver Status:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

Set parameter Username
Set parameter LicenseID to value 2703329
Academic license - for non-commercial use only - expires 2026-09-04
Read LP format model from file C:\Users\macar\AppData\Local\Temp\tmpeqatq1ei.pyomo.lp
Reading time = 0.02 seconds
x1: 28 rows, 28 columns, 103 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28 rows, 28 columns and 103 nonzeros
Model fingerprint: 0xf20090e0
Variable types: 13 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 13650.000000
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 28 rows, 26 columns, 101 nonzeros
Variable types

Qué producto se fabrica en cada línea

In [43]:
# 1. Qué producto se fabrica en cada línea
print("Asignación de producto por línea:")
for l in model.l:
    for e in model.e:
        if pe.value(model.z[e, l]) > 0.5:  # Si está asignado (binaria)
            print(f"  Línea {l} fabrica el producto {e}")


Asignación de producto por línea:
  Línea L1 fabrica el producto C
  Línea L2 fabrica el producto C
  Línea L3 fabrica el producto C


Qué robot está instalado en cada línea

In [44]:

print("Robots instalados por línea:")
for l in model.l:
    robots_en_linea = [r for r in model.r if pe.value(model.u[r, l]) > 0.5]
    if robots_en_linea:
        for r in robots_en_linea:
            print(f"  Robot {r} instalado en línea {l}")
    else:
        print(f"  No hay robots instalados en línea {l}")


Robots instalados por línea:
  Robot R1 instalado en línea L1
  Robot R2 instalado en línea L2
  No hay robots instalados en línea L3


Producción por línea y producto

In [45]:

print("Producción por línea y producto:")
for l in model.l:
    for e in model.e:
        prod = pe.value(model.x[e, l])
        if prod > 1e-3:  # Si se ha producido algo
            print(f"  Línea {l}, producto {e}: {prod:.2f} unidades")

Producción por línea y producto:
  Línea L1, producto C: 500.00 unidades
  Línea L2, producto C: 550.00 unidades
  Línea L3, producto C: 400.00 unidades


Beneficio total obtenido (restando coste de robots)

In [46]:
beneficio = sum(pe.value(model.P[e]) * pe.value(model.x[e, l]) for e in model.e for l in model.l)
coste_robots = sum(pe.value(model.C[r]) * pe.value(model.u[r, l]) for r in model.r for l in model.l)
beneficio_neto = beneficio - coste_robots

print(f"Beneficio total bruto: {beneficio:.2f} €")
print(f"Coste total de robots: {coste_robots:.2f} €")
print(f"Beneficio neto: {beneficio_neto:.2f} €")

Beneficio total bruto: 14500.00 €
Coste total de robots: 2400.00 €
Beneficio neto: 12100.00 €


Total de productos producidos (todas líneas y productos)


In [47]:
total_producido = sum(pe.value(model.x[e, l]) for e in model.e for l in model.l)
print(f"Total de unidades producidas: {total_producido:.2f}")

Total de unidades producidas: 1450.00
